In [14]:
library(devtools)
devtools::install_github("skinnider/dismay", lib="C:/Users/uqewats6/Miniconda3/envs/r_prox/lib/R/library", force=TRUE)
BiocManager::install("WGCNA",force = TRUE, lib="C:/Users/uqewats6/Miniconda3/envs/r_prox/lib/R/library", dependencies=TRUE) 

iterators   (1.0.13 -> 1.0.14) [CRAN]
foreach     (1.5.1  -> 1.5.2 ) [CRAN]
crayon      (1.4.2  -> 1.5.1 ) [CRAN]
cli         (3.1.0  -> 3.2.0 ) [CRAN]
colorspace  (2.0-2  -> 2.0-3 ) [CRAN]
pillar      (1.6.4  -> 1.7.0 ) [CRAN]
fansi       (0.5.0  -> 1.0.3 ) [CRAN]
stringi     (1.7.5  -> 1.7.6 ) [CRAN]
xfun        (0.28   -> 0.30  ) [CRAN]
evaluate    (0.14   -> 0.15  ) [CRAN]
yaml        (2.2.1  -> 2.3.5 ) [CRAN]
jsonlite    (1.7.2  -> 1.8.0 ) [CRAN]
backports   (1.3.0  -> 1.4.1 ) [CRAN]
magrittr    (2.0.1  -> 2.0.2 ) [CRAN]
knitr       (1.36   -> 1.38  ) [CRAN]
rlang       (0.4.12 -> 1.0.2 ) [CRAN]
glue        (1.5.0  -> 1.6.2 ) [CRAN]
digest      (0.6.28 -> 0.6.29) [CRAN]
htmlTable   (2.3.0  -> 2.4.0 ) [CRAN]
rbibutils   (2.2.4  -> 2.2.7 ) [CRAN]
doParallel  (1.0.16 -> 1.0.17) [CRAN]
plyr        (1.8.6  -> 1.8.7 ) [CRAN]
igraph      (1.2.8  -> 1.2.11) [CRAN]
poorman     (NA     -> 0.2.5 ) [CRAN]
generics    (0.1.1  -> 0.1.2 ) [CRAN]
Rdpack      (2.1.2  -> 2.3   ) [CRAN]
philentropy 

Skipping 4 packages not available: AnnotationDbi, GO.db, preprocessCore, impute

Installing 28 packages: iterators, foreach, crayon, cli, colorspace, pillar, fansi, stringi, xfun, evaluate, yaml, jsonlite, backports, magrittr, knitr, rlang, glue, digest, htmlTable, rbibutils, doParallel, plyr, igraph, poorman, generics, Rdpack, philentropy, arules



package 'iterators' successfully unpacked and MD5 sums checked
package 'foreach' successfully unpacked and MD5 sums checked
package 'crayon' successfully unpacked and MD5 sums checked
package 'cli' successfully unpacked and MD5 sums checked
package 'colorspace' successfully unpacked and MD5 sums checked
package 'pillar' successfully unpacked and MD5 sums checked
package 'fansi' successfully unpacked and MD5 sums checked
package 'stringi' successfully unpacked and MD5 sums checked
package 'xfun' successfully unpacked and MD5 sums checked
package 'evaluate' successfully unpacked and MD5 sums checked
package 'yaml' successfully unpacked and MD5 sums checked
package 'jsonlite' successfully unpacked and MD5 sums checked
package 'backports' successfully unpacked and MD5 sums checked
package 'magrittr' successfully unpacked and MD5 sums checked
package 'knitr' successfully unpacked and MD5 sums checked
package 'rlang' successfully unpacked and MD5 sums checked
package 'glue' successfully unpa

In [2]:
library('Seurat')
library('WGCNA')
library('dismay')

Attaching SeuratObject

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: 'fastcluster'


The following object is masked from 'package:stats':

    hclust





Attaching package: 'WGCNA'


The following object is masked from 'package:stats':

    cor



Attaching package: 'dismay'


The following object is masked from 'package:stats':

    binomial




Options for proximity metrics in R are:

- phi_s
- rho_p
- weighted_rank
- zi-kendall

From the implementation provided by the dismay package (https://github.com/skinnider/dismay). As dismay package computes metrics as simialrities, they are converted to dissimilarities when merged with python metrics. 

In [4]:
data_dir = 'C:/Users/uqewats6/PycharmProjects/DimensionalityReduction_Aim_1/Data/'

datasets = c("Discrete_Rare_Processed","Discrete_Abundant_Processed",
             "Continuous_Rare_Processed","Continuous_Abundant_Processed")

metrics = c('kendall') # 'phi_s', 'rho_p', 'weighted_rank','zi_kendall'


Compute a distance matrix for each metric listed on each dataset provided, and outputs them to the data directory as .csv files for integration into the anndata object in python.

In [5]:
for (data in datasets){
  dat <- readRDS(paste(data_dir,data,".rds",sep =""))
  x <- as.matrix(dat@assays$RNA@data)
  for (m in metrics){
    out = dismay(x, m)
    nm = paste(data,m,'.csv', sep="_")
    write.csv(out, file=paste(data_dir, nm, sep=""))
  }
}